In [1]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
import zipfile
import tempfile

In [2]:
data = pd.read_csv(r"/Users/ankitakhatri/Downloads/EQR_2019_Q4_data.csv")
data

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,class_name,term_name,increment_name,increment_peaking_name,transaction_quantity,price,rate_units,standardized_quantity,standardized_price,total_transaction_charge
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,201910010000,202001010000,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,F,LT,Y,FP,1848.30,18675.000000,$/MW-MO,1848.30,18675.000000,34517002.50
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,201910010000,201910312359,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,117.28,10447.558000,$/MW-MO,117.28,10447.558000,1225289.55
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,201911010000,201911302359,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,117.78,10447.557600,$/MW-MO,117.78,10447.557600,1230513.33
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,201912010000,201912312359,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,120.05,10447.557600,$/MW-MO,120.05,10447.557600,1254229.29
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",201910010000,201911010000,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,F,LT,Y,FP,525.00,4061.988571,$/MW-MO,525.00,4061.988571,2132544.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,T2,CalPeak Power - Border LLC,CalPeak Power LLC,201911010000,201911302359,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,28.00,2991.800000,$/MW-MO,28.00,2991.800000,83770.40
1715,T3,CalPeak Power - Border LLC,CalPeak Power LLC,201912010000,201912312359,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,28.00,3312.350000,$/MW-MO,28.00,3312.350000,92745.80
1716,T5,CalPeak Power - Border LLC,CalPeak Power LLC,201910010000,201910312359,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00
1717,T6,CalPeak Power - Border LLC,CalPeak Power LLC,201911010000,201911302359,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00


In [3]:
data_deleted_data = pd.DataFrame(columns=data.columns)

In [4]:
# create new df to do all the cleaning on and convert stuff to datetime 
data_clean = data.copy()
data_clean['transaction_end_date'] = pd.to_datetime(data_clean['transaction_end_date'], format = '%Y%m%d%H%M') # convert to datetime 
data_clean['transaction_begin_date'] = pd.to_datetime(data_clean['transaction_begin_date'], format = '%Y%m%d%H%M') # convert to datetime 
data_clean.loc[data_clean['transaction_end_date'].dt.minute == 0, 'transaction_end_date'] -= pd.Timedelta(minutes = 1) # change everything to end in 59 minutes for consistency
data_clean

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,class_name,term_name,increment_name,increment_peaking_name,transaction_quantity,price,rate_units,standardized_quantity,standardized_price,total_transaction_charge
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,2019-10-01,2019-12-31 23:59:00,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,F,LT,Y,FP,1848.30,18675.000000,$/MW-MO,1848.30,18675.000000,34517002.50
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,117.28,10447.558000,$/MW-MO,117.28,10447.558000,1225289.55
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,117.78,10447.557600,$/MW-MO,117.78,10447.557600,1230513.33
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,UP,LT,Y,FP,120.05,10447.557600,$/MW-MO,120.05,10447.557600,1254229.29
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,F,LT,Y,FP,525.00,4061.988571,$/MW-MO,525.00,4061.988571,2132544.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,28.00,2991.800000,$/MW-MO,28.00,2991.800000,83770.40
1715,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,28.00,3312.350000,$/MW-MO,28.00,3312.350000,92745.80
1716,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00
1717,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,F,LT,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00


In [5]:
data_clean['converted_price'] = pd.NA
data_clean['converted_rate_unit'] = pd.NA

In [6]:
# drop any rows where price is NA 
na_rows = data_clean[data_clean['price'].isna()]
data_deleted_data = pd.concat([data_deleted_data, na_rows], ignore_index=True)
data_clean = data_clean.dropna(subset=['price']).reset_index(drop=True)
print(len(data_clean))
data_clean

1719


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,increment_name,increment_peaking_name,transaction_quantity,price,rate_units,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,2019-10-01,2019-12-31 23:59:00,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,...,Y,FP,1848.30,18675.000000,$/MW-MO,1848.30,18675.000000,34517002.50,NaN,NaN
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.28,10447.558000,$/MW-MO,117.28,10447.558000,1225289.55,NaN,NaN
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.78,10447.557600,$/MW-MO,117.78,10447.557600,1230513.33,NaN,NaN
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,120.05,10447.557600,$/MW-MO,120.05,10447.557600,1254229.29,NaN,NaN
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,Y,FP,525.00,4061.988571,$/MW-MO,525.00,4061.988571,2132544.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,2991.800000,$/MW-MO,28.00,2991.800000,83770.40,NaN,NaN
1715,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,3312.350000,$/MW-MO,28.00,3312.350000,92745.80,NaN,NaN
1716,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,NaN,NaN
1717,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,NaN,NaN


In [7]:
# price conversions to $/KW-MO for corresponding rate units, anything that is $/MW is deleted because it is frequency response 
for index, row in data.iterrows():
    price = row['price']
    rate_unit = row['rate_units']

    if rate_unit =='$/KW-MO':
        data_clean.at[index, 'converted_price'] = price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/MW-MO':
        converted_price = price / 1000
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'
    
    elif rate_unit == '$/MW-DAY':
        converted_price = (price * 365 / 12) / 1000
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/KW-DAY':
        converted_price = price * (365 / 12)
        data_clean.at[index, 'converted_price'] = converted_price
        data_clean.at[index, 'converted_rate_unit'] = '$/KW-MO'

    elif rate_unit == '$/MW':
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        data_clean.drop(index, inplace=True)
    
    elif rate_unit == '$/KW':
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        data_clean.drop(index, inplace=True)

data_clean

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,increment_name,increment_peaking_name,transaction_quantity,price,rate_units,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,2019-10-01,2019-12-31 23:59:00,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,...,Y,FP,1848.30,18675.000000,$/MW-MO,1848.30,18675.000000,34517002.50,18.675,$/KW-MO
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.28,10447.558000,$/MW-MO,117.28,10447.558000,1225289.55,10.447558,$/KW-MO
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.78,10447.557600,$/MW-MO,117.78,10447.557600,1230513.33,10.447558,$/KW-MO
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,120.05,10447.557600,$/MW-MO,120.05,10447.557600,1254229.29,10.447558,$/KW-MO
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,Y,FP,525.00,4061.988571,$/MW-MO,525.00,4061.988571,2132544.00,4.061989,$/KW-MO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,2991.800000,$/MW-MO,28.00,2991.800000,83770.40,2.9918,$/KW-MO
1715,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,3312.350000,$/MW-MO,28.00,3312.350000,92745.80,3.31235,$/KW-MO
1716,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,3.5,$/KW-MO
1717,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,3.5,$/KW-MO


In [8]:
# drop any rows where converted price is NA 
na_rows = data_clean[data_clean['converted_price'].isna()]
data_deleted_data = pd.concat([data_deleted_data, na_rows], ignore_index=True)
data_clean = data_clean.dropna(subset=['converted_price']).reset_index(drop=True)
print(len(data_clean))
data_clean

1703


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,increment_name,increment_peaking_name,transaction_quantity,price,rate_units,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,2019-10-01,2019-12-31 23:59:00,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,...,Y,FP,1848.30,18675.000000,$/MW-MO,1848.30,18675.000000,34517002.50,18.675,$/KW-MO
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.28,10447.558000,$/MW-MO,117.28,10447.558000,1225289.55,10.447558,$/KW-MO
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,117.78,10447.557600,$/MW-MO,117.78,10447.557600,1230513.33,10.447558,$/KW-MO
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,Y,FP,120.05,10447.557600,$/MW-MO,120.05,10447.557600,1254229.29,10.447558,$/KW-MO
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,Y,FP,525.00,4061.988571,$/MW-MO,525.00,4061.988571,2132544.00,4.061989,$/KW-MO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,2991.800000,$/MW-MO,28.00,2991.800000,83770.40,2.9918,$/KW-MO
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,28.00,3312.350000,$/MW-MO,28.00,3312.350000,92745.80,3.31235,$/KW-MO
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,3.5,$/KW-MO
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,Y,FP,20.00,3500.000000,$/MW-MO,20.00,3500.000000,70000.00,3.5,$/KW-MO


In [9]:
# NA value check 
index = 0
for price in data_clean['converted_price']:
    index +=1
    if pd.isna(price) == True:
        print(f"NA converted price index: {index}")

In [10]:
# take out 0 and less
rows_to_delete = []

for index, row in data_clean.iterrows():
    if row['converted_price'] <= 0:
        rows_to_delete.append(index)

rows_to_append = data_clean.loc[rows_to_delete]
data_clean.drop(rows_to_delete, inplace=True)
data_deleted_data = pd.concat([data_deleted_data, rows_to_append], ignore_index = True)

In [11]:
# organize delivery begin and end date columns 
data_clean['begin_date'] = data_clean['transaction_begin_date'].copy()
data_clean['end_date'] = data_clean['transaction_end_date'].copy()

data_clean['date_difference'] = data_clean['end_date'] - data_clean['begin_date']
data_clean['transaction_days'] = data_clean['date_difference'] + timedelta(minutes = 1)

# data_clean['delivery_month'] = data_clean['begin_date'].dt.month
data_clean['delivery_month'] = data_clean['begin_date'].dt.strftime('%Y-%m')

data_clean

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
0,T70,"Russell City Energy Company, LLC",Pacific Gas and Electric Company,2019-10-01,2019-12-31 23:59:00,20100409.0,Fixed,PP,CISO,RUSCTY_2_UNITS-APND,...,1848.30,18675.000000,34517002.50,18.675,$/KW-MO,2019-10-01,2019-12-31 23:59:00,91 days 23:59:00,92 days,2019-10
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.28,10447.558000,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.78,10447.557600,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,120.05,10447.557600,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,2991.800000,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,3312.350000,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11


In [12]:
# delete any transactions that are shorter or longer than a month 
deleted_indices = []

for index, row in data_clean.iterrows():
    days = row['transaction_days'].days
    if days < 25:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        deleted_indices.append(index)

    elif days > 31:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        deleted_indices.append(index)

data_clean = data_clean.drop(deleted_indices)

print(len(data_clean))
data_clean

1404


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.28,10447.558000,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.78,10447.557600,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,120.05,10447.557600,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
5,T2,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-11-01,2019-11-30 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,2991.800000,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,3312.350000,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11


In [13]:
print(data_clean['converted_price'].mean())
print(data_clean['converted_price'].std())
print(data_clean['converted_price'].min())
print(data_clean['converted_price'].max())
data_clean

3.5741021374893216
5.198401658748147
0.0004395
97.18335938


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.28,10447.558000,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.78,10447.557600,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,120.05,10447.557600,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
5,T2,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-11-01,2019-11-30 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,2991.800000,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,3312.350000,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11


In [14]:
average = data_clean['converted_price'].mean()
stdev = data_clean['converted_price'].std()
print(average)
print(stdev)

3.5741021374893216
5.198401658748147


In [15]:
# remove less than 1 normal stdevs 
threshold = average - stdev
print(threshold)

indices_to_delete = []
for index, row in data_clean.iterrows():
    converted_price = row['converted_price']
    # print(f"Converted Price: {converted_price}")
    if converted_price < threshold:
        data_deleted_data = pd.concat([data_deleted_data, row.to_frame().T], ignore_index=True)
        indices_to_delete.append(index)

data_clean = data_clean.drop(indices_to_delete)

print(len(data_clean))
data_clean

-1.6242995212588256
1404


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.28,10447.558000,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.78,10447.557600,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,120.05,10447.557600,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
5,T2,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-11-01,2019-11-30 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,2991.800000,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,3312.350000,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11


In [16]:
print(len(indices_to_delete))

0


In [17]:
print(data_clean['converted_price'].min())
print(data_clean['converted_price'].max())

0.0004395
97.18335938


In [18]:
i = 0
for price in data_clean['converted_price']:
    i +=1
    if pd.isna(price) == True:
        print(f"NA converted price index: {index}")

In [19]:
print(len(data_clean))
data_clean

1404


,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.28,10447.558000,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,117.78,10447.557600,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,20060403.0,Formula,PP,CISO,PNCHPP_1_PL1X2,...,120.05,10447.557600,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
5,T2,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-11-01,2019-11-30 23:59:00,20150101.0,Fixed,PP,CISO,SUTTER_2_PL1X3,...,525.00,4061.988571,2132544.00,4.061989,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,2991.800000,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,20180727.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,28.00,3312.350000,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,20180711.0,FIXED,PP,CISO,BORDER_6_UNITA1,...,20.00,3500.000000,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11


In [20]:
data_clean['product'] = data_clean['standardized_quantity'] * data_clean['converted_price']
sumproduct = data_clean['product'].sum()
print(sumproduct)
sum_weights = data_clean['standardized_quantity'].sum()
weighted_avg = sumproduct / sum_weights
print(f"Weighted Average: {weighted_avg}")

diff_squared = (data_clean['converted_price'] - weighted_avg) ** 2
sumproduct_diff = np.sum(diff_squared * data_clean['standardized_quantity'])
weighted_std_dev = np.sqrt(sumproduct_diff / (sum_weights - 1))
print(f"Weighted Std Dev: {weighted_std_dev}")

4990149.724045834
Weighted Average: 0.6106191237421515
Weighted Std Dev: 1.5334304708855777


In [21]:
# convert trade date to datetime then extract into new column the formatting wanted
data_clean['trade_date'] = pd.to_datetime(data_clean['trade_date'], format = '%Y%m%d')
data_clean['trade_date_year_mo'] = data_clean['trade_date'].dt.strftime('%Y-%m')
data_clean

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month,product,trade_date_year_mo
1,T4,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-10-01,2019-10-31 23:59:00,2006-04-03,Formula,PP,CISO,PNCHPP_1_PL1X2,...,1225289.55,10.447558,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10,1225.289602,2006-04
2,T5,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-11-01,2019-11-30 23:59:00,2006-04-03,Formula,PP,CISO,PNCHPP_1_PL1X2,...,1230513.33,10.447558,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11,1230.513334,2006-04
3,T6,"Midway Peaking, LLC",Pacific Gas & Electric Company,2019-12-01,2019-12-31 23:59:00,2006-04-03,Formula,PP,CISO,PNCHPP_1_PL1X2,...,1254229.29,10.447558,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12,1254.22929,2006-04
4,T1,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-10-01,2019-10-31 23:59:00,2015-01-01,Fixed,PP,CISO,SUTTER_2_PL1X3,...,2132544.00,4.061989,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10,2132.544,2015-01
5,T2,"CCFC Sutter Energy, LLC","Calpine Energy Services, L.P.",2019-11-01,2019-11-30 23:59:00,2015-01-01,Fixed,PP,CISO,SUTTER_2_PL1X3,...,2132544.00,4.061989,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11,2132.544,2015-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,T2,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,2018-07-27,FIXED,PP,CISO,BORDER_6_UNITA1,...,83770.40,2.9918,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11,83.7704,2018-07
1699,T3,CalPeak Power - Border LLC,CalPeak Power LLC,2019-12-01,2019-12-31 23:59:00,2018-07-27,FIXED,PP,CISO,BORDER_6_UNITA1,...,92745.80,3.31235,$/KW-MO,2019-12-01,2019-12-31 23:59:00,30 days 23:59:00,31 days,2019-12,92.7458,2018-07
1700,T5,CalPeak Power - Border LLC,CalPeak Power LLC,2019-10-01,2019-10-31 23:59:00,2018-07-11,FIXED,PP,CISO,BORDER_6_UNITA1,...,70000.00,3.5,$/KW-MO,2019-10-01,2019-10-31 23:59:00,30 days 23:59:00,31 days,2019-10,70.0,2018-07
1701,T6,CalPeak Power - Border LLC,CalPeak Power LLC,2019-11-01,2019-11-30 23:59:00,2018-07-11,FIXED,PP,CISO,BORDER_6_UNITA1,...,70000.00,3.5,$/KW-MO,2019-11-01,2019-11-30 23:59:00,29 days 23:59:00,30 days,2019-11,70.0,2018-07


In [22]:
date_range = pd.date_range(start= '2013-01-01', end= '2024-12-01', freq= 'MS')
trade_date_year_mo = date_range.strftime('%Y-%m')
output_df = pd.DataFrame()

for delivery_month in data_clean['delivery_month'].unique():
    df = pd.DataFrame({'trade_date_year_mo': trade_date_year_mo})

    weighted_avg_prices = []
    total_quantities = []

    for year_mo in trade_date_year_mo:
        mask = (data_clean['trade_date_year_mo'] == year_mo) & (data_clean['delivery_month'] == delivery_month)
        filter_data = data_clean[mask]

        weighted_avg_price = 0
        total_quantity = 0

        if not filter_data.empty:
            total_quantity = filter_data['standardized_quantity'].sum()
            if total_quantity > 0:
                weighted_avg_price = (filter_data['standardized_quantity'] * filter_data['converted_price']).sum() / total_quantity
            else:
                weighted_avg_price = float('nan')
        else:
            weighted_avg_price = float('nan')
            total_quantity = float('nan')

        weighted_avg_prices.append(weighted_avg_price)
        # print(weighted_avg_prices)
        total_quantities.append(total_quantity)
        # print(total_quantities)
    print(f"delivery month: {delivery_month}")
    df['delivery_year_mo'] = delivery_month
    df['weighted_avg_price'] = weighted_avg_prices
    df['total_transacted_quantity'] = total_quantities
    print(df)
    

    output_df = pd.concat([output_df, df], ignore_index= True)
    print(f"output df length: {len(output_df)}")
    output_df

delivery month: 2019-10
    trade_date_year_mo delivery_year_mo  weighted_avg_price  \
0              2013-01          2019-10                 NaN   
1              2013-02          2019-10                 NaN   
2              2013-03          2019-10                 NaN   
3              2013-04          2019-10                 NaN   
4              2013-05          2019-10                 NaN   
..                 ...              ...                 ...   
139            2024-08          2019-10                 NaN   
140            2024-09          2019-10                 NaN   
141            2024-10          2019-10                 NaN   
142            2024-11          2019-10                 NaN   
143            2024-12          2019-10                 NaN   

     total_transacted_quantity  
0                          NaN  
1                          NaN  
2                          NaN  
3                          NaN  
4                          NaN  
..                   

In [23]:
data_deleted_data

,transaction_unique_id,seller_company_name,customer_company_name,transaction_begin_date,transaction_end_date,trade_date,type_of_rate,time_zone,point_of_delivery_balancing_authority,point_of_delivery_specific_location,...,standardized_quantity,standardized_price,total_transaction_charge,converted_price,converted_rate_unit,begin_date,end_date,date_difference,transaction_days,delivery_month
0,T57,Powerex Corp.,Sacramento Municipal Utility District,201910010000,201912312359,20181030.0,Fixed,PP,CISO,CAISO Import Capability-Resource Adequacy,...,54.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T58,Powerex Corp.,Sacramento Municipal Utility District,201910010000,201910312359,20181030.0,Fixed,PP,CISO,Resource Adequacy California-Oregon Border,...,18.0,4999.9776,89999.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T141149,Powerex Corp.,Sacramento Municipal Utility District,201911010000,201911302359,20181030.0,Fixed,PP,CISO,Resource Adequacy California-Oregon Border,...,18.0,4999.968,89999.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T62096,"Elk Hills Power, LLC",Southern California Edison Company,201910010000,201910310000,20191031.0,FIXED,PP,CISO,ELKHIL_2_PL1X3,...,80.0,18.31,1464.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T62097,"Elk Hills Power, LLC",Southern California Edison Company,201912310000,201912310000,20191231.0,FIXED,PP,CISO,ELKHIL_2_PL1X3,...,80.0,18.31,1464.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,T498549347,"Exelon Generation Company, LLC",CONSTELLATION NEWENERGY INC,2019-10-01 00:00:00,2019-12-31 23:59:00,20190429.0,Fixed,CP,CISO,Greater Bay,...,54.0,12950.0,699300.0,12.95,$/KW-MO,2019-10-01 00:00:00,2019-12-31 23:59:00,91 days 23:59:00,92 days 00:00:00,2019-10
311,T498549348,"Exelon Generation Company, LLC",CONSTELLATION NEWENERGY INC,2019-10-01 00:00:00,2019-12-31 23:59:00,20180413.0,Fixed,CP,CISO,Greater Bay,...,150.0,2750.0,412500.0,2.75,$/KW-MO,2019-10-01 00:00:00,2019-12-31 23:59:00,91 days 23:59:00,92 days 00:00:00,2019-10
312,T498549349,"Exelon Generation Company, LLC",CONSTELLATION NEWENERGY INC,2019-10-01 00:00:00,2019-12-31 23:59:00,20180413.0,Fixed,CP,CISO,System,...,210.0,2850.0,598500.0,2.85,$/KW-MO,2019-10-01 00:00:00,2019-12-31 23:59:00,91 days 23:59:00,92 days 00:00:00,2019-10
313,T498549350,"Exelon Generation Company, LLC",CONSTELLATION NEWENERGY INC,2019-10-01 00:00:00,2019-12-31 23:59:00,20190131.0,Fixed,CP,CISO,System,...,43.5,3500.0,152250.0,3.5,$/KW-MO,2019-10-01 00:00:00,2019-12-31 23:59:00,91 days 23:59:00,92 days 00:00:00,2019-10


In [24]:
data_clean_csv_path = r"/Users/ankitakhatri/Downloads/EQR_2019_Q4_clean.csv"
data_clean.to_csv(data_clean_csv_path, index=False)

In [25]:
output_df_csv_path = r"/Users/ankitakhatri/Downloads/output_df_2019_Q4.csv"
output_df.to_csv(output_df_csv_path, index=False)

In [26]:
data_deleted_data_csv_path = r"/Users/ankitakhatri/Downloads/data_deleted_data_2019_Q4.csv"
data_deleted_data.to_csv(data_deleted_data_csv_path, index=False)